#사전 작업

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#데이터 전처리


In [27]:
#import 및 파일 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, classification_report
from scipy import stats

plt.rc('font', family='NanumGothic')

file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')



In [28]:
# 숫자 추출 함수 생성
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

# 문자 데이터 제거 및 숫자만 추출
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)

# 결측치 처리
df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)
df['피해운전자 차종'].fillna('차량단독사고', inplace=True)
df['피해운전자 성별'].fillna('차량단독사고', inplace=True)
df['피해운전자 상해정도'].fillna('상해없음', inplace=True)


In [29]:
# 피쳐 생성
df['ECLO'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1

# '사고일시' 열에서 시간 정보 추출
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
df['사고발생시간'] = df['사고일시'].dt.hour

# '시군구' 피처에서 '구' 정보만 남기기
df['시군구'] = df['시군구'].apply(lambda x: x.split(' ')[1])

In [30]:
#범주형 데이터 처리


In [31]:
# 이상치 제거
z_scores = np.abs(stats.zscore(df['ECLO']))
outliers = (z_scores > 3)
df_no_outliers = df[~outliers]


In [ ]:
#스케일링
eclo_data = df_no_outliers['ECLO'].values.reshape(-1, 1)
scaler = StandardScaler()
eclo_scaled = scaler.fit_transform(eclo_data)
df_no_outliers['ECLO_scaled'] = eclo_scaled


#모델링

In [33]:
#회귀 모델 예시

# 피처 선택
features = ['경상자수', '중상자수', '사망자수']

# 독립변수와 종속변수 분할
X = df_no_outliers[features]
y = df_no_outliers['ECLO_scaled']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#모델 훈련
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

# 예측
y_pred_train_rf = rf_regressor.predict(X_train)
y_pred_test_rf = rf_regressor.predict(X_test)

# 성능 평가
print("Random Forest Regressor 성능 평가:")
print("훈련 세트 R2 점수:", r2_score(y_train, y_pred_train_rf))
print("테스트 세트 R2 점수:", r2_score(y_test, y_pred_test_rf))
print("훈련 세트 RMSE:", mean_squared_error(y_train, y_pred_train_rf, squared=False))
print("테스트 세트 RMSE:", mean_squared_error(y_test, y_pred_test_rf, squared=False))
print("훈련 세트 MAE:", mean_absolute_error(y_train, y_pred_train_rf))
print("테스트 세트 MAE:", mean_absolute_error(y_test, y_pred_test_rf))

# 피처 중요도 출력
print("\nRandom Forest Regressor 피처 중요도:")
rf_feature_importances = rf_regressor.feature_importances_
for i, feature in enumerate(features):
    print(f"{feature}: {rf_feature_importances[i]}")


Random Forest Regressor 성능 평가:
훈련 세트 R2 점수: 0.9900442586618505
테스트 세트 R2 점수: 0.9881970895183898
훈련 세트 RMSE: 0.09994262295832529
테스트 세트 RMSE: 0.10791901376210082
훈련 세트 MAE: 0.03166745233794578
테스트 세트 MAE: 0.031942648591598045

Random Forest Regressor 피처 중요도:
경상자수: 0.578689391967671
중상자수: 0.3507287437603595
사망자수: 0.0705818642719696


In [34]:
#분류형 모델 예시(ECLO 값 분포가 넓지 않아서 따로 범주화 진행하지는 않음)

# 피처 선택
features = ['경상자수', '중상자수', '사망자수']

# 독립변수와 종속변수 분할
X = df_no_outliers[features]
y = df_no_outliers['ECLO']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForestClassifier 모델 훈련
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# 예측
y_pred_train_rf_clf = rf_classifier.predict(X_train)
y_pred_test_rf_clf = rf_classifier.predict(X_test)

# 성능 평가
print("Random Forest Classifier 성능 평가:")
print("훈련 정확도:", accuracy_score(y_train, y_pred_train_rf_clf))
print("테스트 정확도:", accuracy_score(y_test, y_pred_test_rf_clf))

# 피처 중요도 출력
print("\nRandom Forest Classifier 피처 중요도:")
rf_feature_importances = rf_classifier.feature_importances_
for i, feature in enumerate(features):
    print(f"{feature}: {rf_feature_importances[i]}")


Random Forest Classifier 성능 평가:
훈련 정확도: 0.9695893451720311
테스트 정확도: 0.9703053322152617

Random Forest Classifier 피처 중요도:
경상자수: 0.685337384998239
중상자수: 0.30006703521922445
사망자수: 0.01459557978253646


#SUB

In [ ]:
print(df.columns.tolist())

In [ ]:
#원-핫 인코딩 예시
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)
print(encoded_df.columns)


#시각화 관련

In [ ]:
!apt-get install -y fonts-nanum*
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

데이터탐색에 들어갈 자료 생성

In [ ]:
# 빈도표
plt.figure(figsize=(16, 12))
sns.countplot(x='피해운전자 연령', data=df, palette='viridis')
plt.ylabel('빈도')
plt.show()


In [ ]:
# 커널 밀도 추정 그래프
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='사고발생시간', fill=True, color='skyblue')
plt.title('사고발생시간별 분포')
plt.xlabel('사고발생시간')
plt.ylabel('밀도')
plt.show()


시각화에 들어갈 자료 생성

In [ ]:
# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('시군구')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='시군구', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def categorize_age(age):
    if age <= 19:
        return '0~19'
    elif age <= 45:
        return '20~45'
    elif age <= 60:
        return '46~60'
    else:
        return '61 이상'

# 연령을 범주
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)

# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('피해운전자 연령대')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='피해운전자 연령대', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3가지 이상의 피처를 활용한 히트맵

heatmap_data = df.pivot_table(index='사고발생시간', columns='법규위반', values='ECLO', aggfunc='mean')

plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".2f", linewidths=.5)
plt.title('사고 발생시간과 법규위반에 따른 ECLO 평균')
plt.xlabel('법규위반')
plt.ylabel('사고발생시간')
plt.show()
